# SQL Assignment - uc288

Found a Python 2 and 3 compatibility cheat sheet online: http://python-future.org/compatible_idioms.html

In [1]:
try:
    from urllib.parse import urlparse, urlencode
    from urllib.request import urlopen, Request
    from urllib.error import HTTPError
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode
    from urllib2 import urlopen, Request, HTTPError
    
from io import StringIO
import ast
import pandas as pd

In [2]:
SQL_SOURCE = 'https://uc288.carto.com/api/v2/sql?q='

In [3]:
def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    data = urlencode({'format': format, 'q': query}).encode('utf-8')
    try:
        response = urlopen(source, data)
    except HTTPError as e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return pd.read_csv(response)

## Task 1: Familiarize with SQL Clauses
* Sort data by start_station_id, tripduration (only checking trips with duration <= 3 hours
* Only show the top/last 10 records
* List all unique start_station_id values
* Aggregate functions:
    * Count the number of trips
    * Find the average / min / max trip duration

In [4]:
sql_1 = '''SELECT DISTINCT start_station_id, 
COUNT(start_station_id) AS trip_counts,
AVG(tripduration) AS avg_tripduration,
MIN(tripduration) AS min_tripduration,
MAX(tripduration) AS max_tripduration
FROM citibike
WHERE tripduration <= 10800
GROUP BY start_station_id
ORDER BY start_station_id ASC, avg_tripduration ASC
LIMIT 10
'''

In [5]:
queryCartoDB(sql_1)

,start_station_id,trip_counts,avg_tripduration,min_tripduration,max_tripduration
0,72,114,742.701754,107,2099
1,79,71,659.140845,149,1916
2,82,49,445.346939,138,1581
3,83,35,783.685714,73,2647
4,116,288,543.965278,66,2081
5,119,19,1044.789474,278,2167
6,120,17,979.882353,270,2131
7,127,209,710.846890,83,5776
8,128,247,664.206478,62,2875
9,137,96,697.364583,85,2316


## Working with date/time
* Selecting trips started on Feb 2, 2015 only
* Selecting trips started on the weekends
* What are the average trip duration during weekends?
* Can we do the sam efor weekday?

### Trips started on Feb. 2, 2015 only

In [6]:
sql_2_1 = '''SELECT starttime, stoptime, start_station_id, 
end_station_id, tripduration, usertype, gender, birth_year
FROM citibike
WHERE starttime >= '2015-02-02 00:00:00'
AND starttime <= '2015-02-02 23:59:59'
ORDER BY starttime
'''

In [7]:
queryCartoDB(sql_2_1)

,starttime,stoptime,start_station_id,end_station_id,tripduration,usertype,gender,birth_year
0,2015-02-02 00:02:00+00,2015-02-02 00:05:00+00,442,489,199,Subscriber,1,1992.0
1,2015-02-02 00:03:00+00,2015-02-02 00:10:00+00,326,349,418,Subscriber,2,1964.0
2,2015-02-02 00:04:00+00,2015-02-02 00:09:00+00,279,415,276,Subscriber,1,1974.0
3,2015-02-02 00:06:00+00,2015-02-02 00:13:00+00,496,237,420,Subscriber,2,1992.0
4,2015-02-02 00:07:00+00,2015-02-02 00:12:00+00,477,450,304,Subscriber,1,1968.0
5,2015-02-02 00:08:00+00,2015-02-02 00:18:00+00,467,391,622,Subscriber,1,1979.0
6,2015-02-02 00:09:00+00,2015-02-02 00:15:00+00,434,482,371,Subscriber,1,1963.0
7,2015-02-02 00:11:00+00,2015-02-02 00:19:00+00,330,2008,477,Subscriber,1,1982.0
8,2015-02-02 00:12:00+00,2015-02-02 00:25:00+00,521,423,743,Subscriber,2,1978.0
9,2015-02-02 00:12:00+00,2015-02-02 00:22:00+00,504,453,599,Subscriber,1,1956.0


### Trips that started on the weekends with the average duration

In [8]:
sql_2_2 = '''SELECT date_part('dow', starttime) as day_of_week,
AVG(tripduration) as avg_tripduration
FROM citibike
WHERE date_part('dow', starttime) IN (0, 6)
GROUP BY date_part('dow', starttime)'''

In [9]:
queryCartoDB(sql_2_2)

,day_of_week,avg_tripduration
0,0,638.149200
1,6,686.460825


In [10]:
sql_2_21 = '''SELECT AVG(tripduration) as avg_tripduration
FROM citibike
WHERE date_part('dow', starttime) IN (0, 6)'''

In [11]:
queryCartoDB(sql_2_21)

,avg_tripduration,Unnamed: 1
0,662.942181,NaN


### Trips that started on the weekends with the average duration

In [12]:
sql_2_3 = '''SELECT date_part('dow', starttime) as day_of_week,
AVG(tripduration) as avg_tripduration
FROM citibike
WHERE date_part('dow', starttime) > 0
AND  date_part('dow', starttime) < 6
GROUP BY date_part('dow', starttime)
ORDER BY day_of_week'''

In [13]:
queryCartoDB(sql_2_3)

,day_of_week,avg_tripduration
0,1,920.862234
1,2,767.224443
2,3,697.556559
3,4,623.396720
4,5,637.116968


In [14]:
sql_2_31 = '''SELECT AVG(tripduration) as avg_tripduration
FROM citibike
WHERE date_part('dow', starttime) > 0
AND  date_part('dow', starttime) < 6'''

In [15]:
queryCartoDB(sql_2_31)

,avg_tripduration,Unnamed: 1
0,681.052292,NaN


## Task 3 - Working with Space

In [16]:
sql_3 = '''SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) AS the_geom_webmercator,
    start_station_latitude, start_station_longitude,
    MIN(cartodb_id) AS cartodb_id,
    COUNT(tripduration) AS numTrips
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography,
                500)
GROUP BY start_station_latitude, start_station_longitude
ORDER BY numtrips'''

In [17]:
queryCartoDB(sql_3)

,the_geom_webmercator,start_station_latitude,start_station_longitude,cartodb_id,numtrips
0,0101000020110F0000F22A632FBE6A5FC1363A28CFADFB...,40.755273,-73.983169,1115,112
1,0101000020110F00009BB87D02B76A5FC146C144E9E5FB...,40.756800,-73.982912,19,141
2,0101000020110F00001607D538556B5FC119A6CA6F41FC...,40.759291,-73.988597,124,141
3,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,40.760647,-73.984427,42,213
4,0101000020110F00003F9300B2976B5FC1B8B88F3102FC...,40.757570,-73.990985,783,221
5,0101000020110F000004AF971C1D6B5FC17A48F3C1A8FB...,40.755136,-73.986580,54,251
6,0101000020110F0000F55695027D6B5FC186E5BB69D7FB...,40.756405,-73.990026,33,507


## Task 4 - Putting it all together

### Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare! 

In [18]:
sql_4_1 = '''SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) AS the_geom_webmercator,
    start_station_latitude, start_station_longitude,
    MIN(cartodb_id) AS cartodb_id,
    AVG(tripduration) AS avg_duration
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography,
                500)
AND date_part('dow', starttime) IN (0, 6)
GROUP BY start_station_latitude, start_station_longitude
ORDER BY avg_duration DESC'''

In [19]:
queryCartoDB(sql_4_1)

,the_geom_webmercator,start_station_latitude,start_station_longitude,cartodb_id,avg_duration
0,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,40.760647,-73.984427,42,1010.104167
1,0101000020110F00001607D538556B5FC119A6CA6F41FC...,40.759291,-73.988597,124,762.931818
2,0101000020110F000004AF971C1D6B5FC17A48F3C1A8FB...,40.755136,-73.986580,54,683.121212
3,0101000020110F00009BB87D02B76A5FC146C144E9E5FB...,40.756800,-73.982912,19,675.400000
4,0101000020110F0000F55695027D6B5FC186E5BB69D7FB...,40.756405,-73.990026,33,643.260274
5,0101000020110F0000F22A632FBE6A5FC1363A28CFADFB...,40.755273,-73.983169,1115,629.700000
6,0101000020110F00003F9300B2976B5FC1B8B88F3102FC...,40.757570,-73.990985,783,579.142857


In [20]:
sql_4_11 = '''SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) AS the_geom_webmercator,
    start_station_latitude, start_station_longitude,
    date_part('dow', starttime) AS day_of_week, 
    MIN(cartodb_id) AS cartodb_id,
    AVG(tripduration) AS avg_duration
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography,
                500)
AND date_part('dow', starttime) IN (0, 6)
GROUP BY start_station_latitude, start_station_longitude, date_part('dow', starttime) 
ORDER BY day_of_week, avg_duration DESC'''

In [21]:
queryCartoDB(sql_4_11)

,the_geom_webmercator,start_station_latitude,start_station_longitude,day_of_week,cartodb_id,avg_duration
0,0101000020110F00001607D538556B5FC119A6CA6F41FC...,40.759291,-73.988597,0,124,799.722222
1,0101000020110F000004AF971C1D6B5FC17A48F3C1A8FB...,40.755136,-73.986580,0,54,671.818182
2,0101000020110F0000F55695027D6B5FC186E5BB69D7FB...,40.756405,-73.990026,0,33,647.483871
3,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,40.760647,-73.984427,0,42,594.954545
4,0101000020110F00009BB87D02B76A5FC146C144E9E5FB...,40.756800,-73.982912,0,19,555.000000
5,0101000020110F00003F9300B2976B5FC1B8B88F3102FC...,40.757570,-73.990985,0,783,476.800000
6,0101000020110F0000F22A632FBE6A5FC1363A28CFADFB...,40.755273,-73.983169,0,1115,410.500000
7,0101000020110F000020D6742CE16A5FC112F4D73D73FC...,40.760647,-73.984427,6,39625,1361.384615
8,0101000020110F00009BB87D02B76A5FC146C144E9E5FB...,40.756800,-73.982912,6,41260,813.000000
9,0101000020110F0000F22A632FBE6A5FC1363A28CFADFB...,40.755273,-73.983169,6,40490,775.833333


### Extra: 
**Create lines** for trips started from stations within 500m of Times Square and **lasted less than 2 hours**. The number of trips per each pair of stations are output as attributes of these lines. 

In [22]:
sql_4_2 = '''SELECT ST_Makeline(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)),
                   CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude, end_station_longitude))) AS the_geom_webmercator,
    MIN(cartodb_id) AS cartodb_id,
    COUNT(tripduration) AS numTrips
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography,
                500)
    AND tripduration < 7200
GROUP BY start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude'''

In [23]:
queryCartoDB(sql_4_2)

,the_geom_webmercator,cartodb_id,numtrips
0,0102000020110F000002000000F22A632FBE6A5FC1363A...,12043,1
1,0102000020110F0000020000009BB87D02B76A5FC146C1...,34407,1
2,0102000020110F00000200000004AF971C1D6B5FC17A48...,30188,1
3,0102000020110F0000020000003F9300B2976B5FC1B8B8...,7566,5
4,0102000020110F00000200000020D6742CE16A5FC112F4...,8850,4
5,0102000020110F0000020000003F9300B2976B5FC1B8B8...,31004,1
6,0102000020110F00000200000020D6742CE16A5FC112F4...,4636,2
7,0102000020110F000002000000F55695027D6B5FC186E5...,6788,6
8,0102000020110F000002000000F22A632FBE6A5FC1363A...,20181,2
9,0102000020110F0000020000001607D538556B5FC119A6...,26631,1
